In [6]:
import pandas as pd
from pathlib import Path

In [9]:
current_file = Path().resolve()
models_dir = current_file
while models_dir.name != "models" and models_dir != models_dir.parent:
    models_dir = models_dir.parent
    
csv_name = 'fall_pose_keypoint.csv'
print(models_dir)
csv_path = Path( models_dir / "classifiers" /"rf_classifier" / "pose" / "yolo_pose" / csv_name)
    
df = pd.read_csv(csv_path)
print("loading csv")
df = df.drop('image_name', axis=1)
df.head()

/Users/kobe/Documents/UCLL/School 2024-2025/AI Applications/ai-app-bot-brigade/src/models
loading csv


,label,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
0,not-fall,0.485839,0.410512,0.495773,0.394320,0.477423,0.389139,0.508761,0.388049,0.458084,...,0.436837,0.600978,0.512779,0.606168,0.414417,0.604592,0.465867,0.797919,0.399739,0.786769
1,not-fall,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.598155,0.194260,0.000000,...,0.630579,0.438007,0.598843,0.579984,0.629291,0.577627,0.610139,0.726956,0.633059,0.714906
2,not-fall,0.607777,0.382062,0.630249,0.331423,0.585865,0.330294,0.668024,0.313463,0.554278,...,0.540982,0.965722,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,not-fall,0.597149,0.269850,0.640758,0.237419,0.592207,0.213645,0.726758,0.251189,0.000000,...,0.603001,0.865304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,not-fall,0.320964,0.260311,0.333139,0.227619,0.298050,0.230738,0.000000,0.000000,0.255071,...,0.253200,0.676568,0.510862,0.631657,0.313992,0.694880,0.408492,0.859661,0.215078,0.917820


In [14]:
df.shape

(546, 35)

In [13]:
image_width = 1920 
image_height = 1080 

flattened_data = []

for index, row in df.iterrows():
    label = row['label']

    keypoints = []
    for joint in [
        'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',
        'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
        'left_wrist', 'right_wrist', 'left_hip', 'right_hip',
        'left_knee', 'right_knee', 'left_ankle', 'right_ankle'
    ]:
        x = row[f'{joint}_x']
        y = row[f'{joint}_y']
        
        x_norm = x / image_width if x > 0 else 0
        y_norm = y / image_height if y > 0 else 0
        
        confidence = row.get(f'{joint}_conf', 0)    

        keypoints.extend([x_norm, y_norm, confidence])

    flattened_data.append([label] + keypoints)
num_keypoints = len(flattened_data[0]) - 1 
header = ['label'] + [f'joint_{i}' for i in range(num_keypoints)]
flattened_df = pd.DataFrame(flattened_data, columns=header)

flattened_df.to_csv('fall_pose_keypoint_flattened.csv', index=False)

flattened_df.head()


,label,joint_0,joint_1,joint_2,joint_3,joint_4,joint_5,joint_6,joint_7,joint_8,...,joint_41,joint_42,joint_43,joint_44,joint_45,joint_46,joint_47,joint_48,joint_49,joint_50
0,not-fall,0.000253,0.000380,0,0.000258,0.000365,0,0.000249,0.000360,0,...,0,0.000216,0.000560,0,0.000243,0.000739,0,0.000208,0.000728,0
1,not-fall,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,...,0,0.000328,0.000535,0,0.000318,0.000673,0,0.000330,0.000662,0
2,not-fall,0.000317,0.000354,0,0.000328,0.000307,0,0.000305,0.000306,0,...,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0
3,not-fall,0.000311,0.000250,0,0.000334,0.000220,0,0.000308,0.000198,0,...,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0
4,not-fall,0.000167,0.000241,0,0.000174,0.000211,0,0.000155,0.000214,0,...,0,0.000164,0.000643,0,0.000213,0.000796,0,0.000112,0.000850,0


In [15]:
flattened_df.shape

(546, 52)